**Frequency dressed with -40 dB:** 7.579121159 GHz

In [1]:
%matplotlib widget
import pyvisa
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.optimize import curve_fit


import sys 
sys.path.insert(0, r'C:\Users\oper\SynologyDrive\Lab2023\KIDs\QTLab2324\PAandQubit\Actual_Work')
sys.path.insert(1, r'C:\Users\oper\SynologyDrive\Lab2023\KIDs\QTLab2324\PAandQubit\Instruments\network\RS_Signal_Generator')
sys.path.insert(2, r'C:\Users\oper\SynologyDrive\Lab2023\KIDs\QTLab2324\PAandQubit\Instruments\network\VNA')

import FunFit as FF
import Tuls as Ts
import RS_SMA100B
import NA_N9916A

ip_sg           = '192.168.40.15' 
ip_vna          = '192.168.40.10' 
vna             = NA_N9916A.N9916A(ip = ip_vna)
sGen            = RS_SMA100B.SMA100B(ip = ip_sg)

N9916A:	Connection successful!
SMA100B:	Connection successful!


In [ ]:
f0 = vna.get_freqs()
f0 = np.mean(np.array([int(i) for i in f0.split(',')]))

mode        = 'NA'              
s_par       = 'S21'       
f_span      = 1
f_center    = f0
bw          = 10e3            # IF BANDWIDTH
n_avgs      = 1
n_pts       = 600 
power_lvl   = -40  
                            
###############################

vna.set_num_pts(n_pts)
vna.set_freq_bandwidth(bw)
vna.set_freq_span(f_span)
vna.set_freq_center(f_center)
vna.num_avgs(n_avgs)
vna.power = power_lvl
vna.set_scaling(auto = True)

In [10]:
f_sg_min    = 5.4e9
f_sg_max    = 5.8e9
f_step      = 2e6

lvl_sg_min  = -30
lvl_sg_max  = 2
lvl_step    = 1

freqs_sg    = np.arange(f_sg_min, f_sg_max, f_step)
lvl_sg      = np.arange(lvl_sg_min, lvl_sg_max, lvl_step)

print('len freqs:\t', len(freqs_sg))
print('len lvl:\t', len(lvl_sg))

len freqs:	 200
len lvl:	 32


In [ ]:



amplitude = []
counter = 0

for p in lvl_sg:
	sGen.RF_lvl_ampl(p)
	sGen.RF_state(1)

	for f in freqs_sg:
		print(f'\r{int(counter*100/len(freqs_sg))}%', end='', flush = True)
		counter += 1
		sGen.RF_freq(f)
		time.sleep(1)
		data = vna.get_data()
		amp = np.mean(5*np.log10(data['Q']**2 + data['I']**2))
		amplitude.append(amp)

sGen.RF_state(0)
amplitude = np.array(amplitude)

fig = plt.figure(figsize = (10,6))
plt.plot(freqs_sg, amplitude)
plt.show()